# **Input params samples** (AWS DeepRacer)

- Author: ai-castle
  - https://www.ai-castle.com
  - edu@ai-castle.com

**[Introduction]**
- This document provides samples to verify the structure of Input params in DeepRacer's reward function.
- To extract Input params, you need to write the reward function in the following format.
  - reward function format
    ```python
    class PrintParams:
        def __init__(self):
            self.waypoints_printed = False
        def print(self, params):
            waypoints = params.pop('waypoints')
            if not self.waypoints_printed:
                self.waypoints_printed = True
                print(f"<<<waypoints_start>>>{waypoints}<<<waypoints_end>>>")
            print(f"<<<params_start>>>{params}<<<params_end>>>")

    print_params = PrintParams()
    
    def reward_function(params):
        # ...
        print_params.print(params)
        return float(reward)
    ```
- After the model training is complete, you need to download the log data and preprocess the output in the robomaker log file.
- This document shows the series of steps required to accomplish this. 

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import tarfile
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
def show_params_dataframe(url, max_rows=100):
    
    # Download the log file
    log_filename = "download_logs.tar.gz"
    urllib.request.urlretrieve(url, log_filename)

    # Unzip the log file
    tar = tarfile.open(log_filename, "r:gz")
    file_infos = tar.getmembers()
    file_names =  [i.get_info()['name'] for i in file_infos]
    tar.extractall('log_unzip')
    tar.close()

    # Identify the Robomaker log file
    iteration_file_names = []
    for i in file_names:
        if i[-13:] == 'robomaker.log':
            robomaker_file_name = i
            break

    robomaker_file_path = 'log_unzip/' + robomaker_file_name

    # Extract the log file
    with open(robomaker_file_path) as f:
        log_txt = f.read()

    # Extract waypoints
    log_split = log_txt.split('<<<waypoints_start>>>')
    waypoints_str, params_txt = log_split[1].split('<<<waypoints_end>>>')
    waypoints = eval(waypoints_str)

    # Extract params
    params_split = params_txt.split('<<<params_start>>>')
    params_split = params_split[1:]
    for i in range(len(params_split)):
        target = params_split[i]
        target = target.split('<<<params_end>>>')[0].replace('\n', '')
        target = eval(target)
        params_split[i] = target

    # Convert params to a dataframe
    df_params = pd.DataFrame(params_split)

    # Add waypoints
    df_params['waypoints'] = str(waypoints)

    display(df_params.iloc[:max_rows])


## [1] Time Trial - Continuous action space

In [3]:
url = "https://github.com/ai-castle/deepracer-lecture-public-data/raw/main/log_data_samples/logs-TT-continuous.tar.gz"
show_params_dataframe(url, max_rows=100)

,all_wheels_on_track,x,y,heading,distance_from_center,projection_distance,progress,steps,speed,steering_angle,track_width,track_length,closest_waypoints,is_left_of_center,is_reversed,closest_objects,objects_location,objects_left_of_center,object_in_camera,objects_speed,objects_heading,objects_distance_from_center,objects_distance,is_crashed,is_offtrack,waypoints
0,True,0.322497,2.691373,-84.006075,0.000000,0.140000,0.605583,2.0,0.784473,-30.000000,1.066684,23.118222,"[0, 1]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
1,True,0.319572,2.678847,-85.027137,0.004253,0.151996,0.657471,3.0,0.588895,-10.536257,1.066585,23.118222,"[1, 2]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
2,True,0.312488,2.659821,-87.260787,0.013949,0.169832,0.734625,4.0,0.500000,24.231568,1.066526,23.118222,"[1, 2]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
3,True,0.316719,2.633208,-86.645489,0.013514,0.196776,0.851172,5.0,0.663647,-22.579592,1.066525,23.118222,"[1, 2]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
4,True,0.319136,2.599557,-86.523703,0.015866,0.230432,0.996756,6.0,0.500000,-12.548987,1.066523,23.118222,"[1, 2]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
5,True,0.312500,2.562091,-88.635591,0.027719,0.266588,1.153152,7.0,0.669653,2.026357,1.066521,23.118222,"[1, 2]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
6,True,0.309112,2.517582,-89.688052,0.037669,0.308501,1.334451,8.0,0.762912,-30.000000,1.066564,23.118222,"[2, 3]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
7,True,0.300979,2.475268,-91.664164,0.053490,0.348580,1.507814,9.0,0.865403,-30.000000,1.066555,23.118222,"[2, 3]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
8,True,0.278040,2.425569,-96.692893,0.085227,0.393177,1.700725,10.0,0.652822,1.650501,1.066560,23.118222,"[2, 3]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
9,True,0.251934,2.378611,-101.476058,0.119571,0.434495,1.879447,11.0,0.600166,-27.052564,1.066565,23.118222,"[2, 3]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."


## [2] Time Trial - Discrete action space

In [4]:
url = "https://github.com/ai-castle/deepracer-lecture-public-data/raw/main/log_data_samples/logs-TT-discrete.tar.gz"
show_params_dataframe(url, max_rows=100)

,all_wheels_on_track,x,y,heading,distance_from_center,projection_distance,progress,steps,speed,steering_angle,track_width,track_length,closest_waypoints,is_left_of_center,is_reversed,closest_objects,objects_location,objects_left_of_center,object_in_camera,objects_speed,objects_heading,objects_distance_from_center,objects_distance,is_crashed,is_offtrack,waypoints
0,True,0.322497,2.691373,-84.006075,0.000000,0.140000,0.605583,2.0,1.0,0.0,1.066684,23.118222,"[0, 1]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
1,True,0.323162,2.678946,-84.165911,0.000685,0.152404,0.659239,3.0,0.5,0.0,1.066581,23.118222,"[1, 2]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
2,True,0.324473,2.658235,-84.367369,0.002308,0.173093,0.748731,4.0,1.0,30.0,1.066526,23.118222,"[1, 2]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
3,True,0.348853,2.611427,-79.657843,0.015228,0.222871,0.964049,5.0,0.5,15.0,1.066524,23.118222,"[1, 2]",True,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
4,True,0.364061,2.590469,-76.928896,0.027328,0.245765,1.063078,6.0,1.0,-30.0,1.066522,23.118222,"[1, 2]",True,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
5,True,0.375629,2.547859,-76.612353,0.032771,0.289580,1.252606,7.0,1.0,-30.0,1.066520,23.118222,"[1, 2]",True,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
6,True,0.372258,2.496618,-80.167929,0.020508,0.340786,1.474103,8.0,0.5,-30.0,1.066554,23.118222,"[2, 3]",True,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
7,True,0.358979,2.441729,-85.330779,0.002696,0.392271,1.696802,9.0,0.5,15.0,1.066560,23.118222,"[2, 3]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
8,True,0.351270,2.387400,-88.330295,0.020324,0.444236,1.921583,10.0,1.0,-15.0,1.066568,23.118222,"[2, 3]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
9,True,0.346121,2.346999,-89.980702,0.034028,0.481638,2.083370,11.0,0.5,-30.0,1.066602,23.118222,"[3, 4]",False,False,"[0, 0]",[],[],False,[],[],[],[],False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."


## [3] Object Avoidance - Continuous action space

In [5]:
url = "https://github.com/ai-castle/deepracer-lecture-public-data/raw/main/log_data_samples/logs-OA-continuous.tar.gz"
show_params_dataframe(url, max_rows=100)

,all_wheels_on_track,x,y,heading,distance_from_center,projection_distance,progress,steps,speed,steering_angle,track_width,track_length,closest_waypoints,is_left_of_center,is_reversed,closest_objects,objects_location,objects_left_of_center,object_in_camera,objects_speed,objects_heading,objects_distance_from_center,objects_distance,is_crashed,is_offtrack,waypoints
0,True,0.322497,2.691373,-84.006075,0.000000,0.140000,0.605583,2.0,0.618596,30.000000,1.066684,23.118222,"[0, 1]",False,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
1,True,0.326084,2.683312,-83.384829,0.002725,0.148391,0.641881,3.0,1.000000,0.194598,1.066699,23.118222,"[0, 1]",True,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
2,True,0.337351,2.664769,-81.130261,0.011363,0.168440,0.728602,4.0,0.637076,30.000000,1.066527,23.118222,"[1, 2]",True,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
3,True,0.354745,2.635282,-78.115526,0.024425,0.200086,0.865491,5.0,0.500000,16.587501,1.066525,23.118222,"[1, 2]",True,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
4,True,0.379049,2.601160,-74.103820,0.043674,0.237294,1.026436,6.0,1.000000,1.742770,1.066523,23.118222,"[1, 2]",True,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
5,True,0.399633,2.565111,-71.728719,0.058968,0.275886,1.193369,7.0,0.500000,5.934579,1.066521,23.118222,"[1, 2]",True,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
6,True,0.423357,2.523710,-69.499393,0.075738,0.323615,1.399828,8.0,0.500000,30.000000,1.066552,23.118222,"[2, 3]",True,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
7,True,0.449644,2.485585,-66.727801,0.094517,0.365946,1.582932,9.0,0.500000,-30.000000,1.066557,23.118222,"[2, 3]",True,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
8,True,0.473710,2.444109,-65.213186,0.110494,0.411158,1.778503,10.0,0.787458,10.583372,1.066562,23.118222,"[2, 3]",True,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.2666581

## [4] Object Avoidance - Discrete action space

In [6]:
url = "https://github.com/ai-castle/deepracer-lecture-public-data/raw/main/log_data_samples/logs-OA-discrete.tar.gz"
show_params_dataframe(url, max_rows=100)

,all_wheels_on_track,x,y,heading,distance_from_center,projection_distance,progress,steps,speed,steering_angle,track_width,track_length,closest_waypoints,is_left_of_center,is_reversed,closest_objects,objects_location,objects_left_of_center,object_in_camera,objects_speed,objects_heading,objects_distance_from_center,objects_distance,is_crashed,is_offtrack,waypoints
0,True,0.322497,2.691373,-84.006075,0.000000,0.140000,0.605583,2.0,1.0,15.0,1.066684,23.118222,"[0, 1]",False,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
1,True,0.325111,2.675614,-83.770775,0.000775,0.155977,0.674694,3.0,0.5,15.0,1.066551,23.118222,"[1, 2]",True,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
2,True,0.329437,2.661007,-83.109786,0.002998,0.171049,0.739887,4.0,1.0,0.0,1.066526,23.118222,"[1, 2]",True,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
3,True,0.336696,2.626443,-82.385562,0.005310,0.206291,0.892330,5.0,0.5,-15.0,1.066525,23.118222,"[1, 2]",True,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
4,True,0.339365,2.584614,-82.970251,0.002053,0.248079,1.073086,6.0,0.5,-30.0,1.066522,23.118222,"[1, 2]",True,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
5,True,0.332956,2.551867,-85.412598,0.008910,0.279595,1.209412,7.0,0.5,-30.0,1.066521,23.118222,"[1, 2]",False,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
6,True,0.319300,2.512398,-89.370637,0.028616,0.315481,1.364641,8.0,1.0,15.0,1.066551,23.118222,"[2, 3]",False,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
7,True,0.309394,2.471688,-91.849378,0.045883,0.353655,1.529769,9.0,1.0,-15.0,1.066556,23.118222,"[2, 3]",False,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
8,True,0.299963,2.419513,-93.779755,0.064804,0.403185,1.744012,10.0,1.0,15.0,1.066561,23.118222,"[2, 3]",False,False,"[2, 0]","[[4.580667582090379, -0.08102574690314249], [7...","[False, True, False]",False,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.26635738416950117, 0.26665811689552354, 0.2...","[5.779591587802326, 11.559115678369446, 17.338...",False,Fal

## [5] Head to Head - Continuous action space

In [7]:
url = "https://github.com/ai-castle/deepracer-lecture-public-data/raw/main/log_data_samples/logs-HH-continuous.tar.gz"
show_params_dataframe(url, max_rows=100)

,all_wheels_on_track,x,y,heading,distance_from_center,projection_distance,progress,steps,speed,steering_angle,track_width,track_length,closest_waypoints,is_left_of_center,is_reversed,closest_objects,objects_location,objects_left_of_center,object_in_camera,objects_speed,objects_heading,objects_distance_from_center,objects_distance,is_crashed,is_offtrack,waypoints
0,True,0.322509,2.691257,-84.007505,2.251325e-07,0.140117,0.606089,2.0,0.674346,11.195516,1.066684,23.118222,"[0, 1]",False,False,"[1, 0]","[[1.2860859034402976, 1.4669759864692626], [0....","[True, False]",False,"[0.5, 0.5]","[-0.1429127479338647, -2.1233372537139013]","[0.2642743947754075, 0.2691827396434763]","[2.026471807644861, 22.113498583551152]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
1,True,0.324044,2.683568,-83.865472,7.228772e-04,0.147923,0.639856,3.0,1.000000,-30.000000,1.066696,23.118222,"[0, 1]",True,False,"[1, 0]","[[1.3110269851021996, 1.463734741514936], [0.2...","[True, False]",False,"[0.5, 0.5]","[-0.11478087652904152, -2.0893580858789513]","[0.26420140655471697, 0.2691423255212271]","[2.05162251133361, 22.15919900095711]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
2,True,0.327989,2.664716,-83.404220,2.086794e-03,0.167172,0.723119,4.0,1.000000,-30.000000,1.066527,23.118222,"[1, 2]",True,False,"[1, 0]","[[1.332013279426054, 1.4615982890174457], [0.2...","[True, False]",False,"[0.5, 0.5]","[-0.08755532491082989, -2.0625641882156565]","[0.2647261271274764, 0.267993241084477]","[2.072710746135082, 22.19596558854169]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
3,True,0.311066,2.627946,-88.421035,1.985235e-02,0.201189,0.870260,5.0,0.764653,13.553562,1.066525,23.118222,"[1, 2]",False,False,"[1, 0]","[[1.3546600899175092, 1.4599813996987014], [0....","[True, False]",False,"[0.5, 0.5]","[-0.054438646613551016, -2.0350055426490337]","[0.26597548559492995, 0.26673144934630433]","[2.095380802595637, 22.21148309375468]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
4,True,0.298077,2.592254,-91.901317,3.774469e-02,0.234692,1.015183,6.0,0.525698,9.588970,1.066523,23.118222,"[1, 2]",False,False,"[1, 0]","[[1.3815647894041645, 1.4590162133856224], [0....","[True, False]",False,"[0.5, 0.5]","[-0.01855852691477234, -2.0039753622681955]","[0.26492656138492904, 0.26841283054985315]","[2.15813068638346, 22.24367040038828]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
5,True,0.301883,2.550856,-90.656923,3.981493e-02,0.276213,1.194784,7.0,0.854265,-23.615043,1.066521,23.118222,"[1, 2]",False,False,"[1, 0]","[[1.4060638605903601, 1.45889448145052], [0.17...","[True, False]",False,"[0.5, 0.5]","[0.007634625402717286, -1.9773539962649611]","[0.2646031113685871, 0.26901560330440255]","[2.182627924753065, 22.28296526191405]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
6,True,0.307672,2.510682,-89.129442,4.036080e-02,0.315015,1.362628,8.0,0.500000,-29.999971,1.066551,23.118222,"[2, 3]",False,False,"[1, 0]","[[1.4339549930905666, 1.4594439749190238], [0....","[True, False]",False,"[0.5, 0.5]","[0.030610340130465624, -1.9488154780613893]","[0.2649229342045452, 0.26848507537855565]","[2.2105226362333723, 22.326277292762917]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
7,True,0.300244,2.473747,-91.004898,5.449346e-02,0.349939,1.513694,9.0,0.561355,-19.021637,1.066555,23.118222,"[2, 3]",False,False,"[1, 0]","[[1.4610275908523573, 1.460504054138122], [0.1...","[True, False]",False,"[0.5, 0.5]","[0.04671077218372466, -1.922781048735425]","[0.2657600651927861, 0.266915333781528]","[2.237603044820227, 22.362603815780382]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
8,True,0.289491,2.442474,-93.462298,7.084643e-02,0.378682,1.638026,10.0,0.965231,-3.108993,1.066559,23.118222,"[2, 3]",False,False,"[1, 0]","[[1.488893482313864, 1.4619741083594062], [0.1...","[True, False]",False,"[0.5, 0.5]","[0.058105874740901294, -1.89752670068317

## [6] Head to Head - Discrete action space

In [8]:
url = "https://github.com/ai-castle/deepracer-lecture-public-data/raw/main/log_data_samples/logs-HH-discrete.tar.gz"
show_params_dataframe(url, max_rows=100)

,all_wheels_on_track,x,y,heading,distance_from_center,projection_distance,progress,steps,speed,steering_angle,track_width,track_length,closest_waypoints,is_left_of_center,is_reversed,closest_objects,objects_location,objects_left_of_center,object_in_camera,objects_speed,objects_heading,objects_distance_from_center,objects_distance,is_crashed,is_offtrack,waypoints
0,True,0.322497,2.691373,-84.006075,0.000000,0.140000,0.605583,2.0,0.5,15.0,1.066684,23.118222,"[0, 1]",False,False,"[1, 0]","[[1.2860859034402976, 1.4669759864692626], [0....","[True, False]",False,"[0.5, 0.5]","[-0.1429127479338647, -2.1233372537139013]","[0.2642743947754075, 0.2691827396434763]","[2.026471807644861, 22.113498583551152]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
1,True,0.324956,2.680216,-83.722422,0.001270,0.151399,0.654891,3.0,1.0,15.0,1.066592,23.118222,"[1, 2]",True,False,"[1, 0]","[[1.3064945441728404, 1.4642698496508364], [0....","[True, False]",False,"[0.5, 0.5]","[-0.12022912387164704, -2.0953403723084345]","[0.26416118994061055, 0.2692627051208922]","[2.047058768996538, 22.151075765201067]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
2,True,0.331527,2.659052,-82.682499,0.004791,0.173279,0.749533,4.0,1.0,0.0,1.066526,23.118222,"[1, 2]",True,False,"[1, 0]","[[1.325544023714381, 1.462194873673666], [0.23...","[True, False]",False,"[0.5, 0.5]","[-0.09630577823182779, -2.070678001640367]","[0.2645028700381916, 0.2684475525391491]","[2.066217877817679, 22.18476736142101]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
3,True,0.338480,2.618512,-82.247327,0.005957,0.214394,0.927383,5.0,1.0,-15.0,1.066524,23.118222,"[1, 2]",True,False,"[1, 0]","[[1.3520786711569226, 1.4601270411813294], [0....","[True, False]",False,"[0.5, 0.5]","[-0.0582922672471351, -2.0380858410863496]","[0.2657947231220793, 0.26670002531210657]","[2.092801605203544, 22.21148309375468]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
4,True,0.342727,2.576380,-82.627701,0.004221,0.256704,1.110396,6.0,1.0,-30.0,1.066522,23.118222,"[1, 2]",True,False,"[1, 0]","[[1.3804303631728285, 1.4590380387019635], [0....","[True, False]",False,"[0.5, 0.5]","[-0.019917089058465783, -2.0052449990325543]","[0.26495772668140705, 0.26835861864852106]","[2.1569964783147744, 22.241823479090225]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
5,True,0.332915,2.515558,-86.691077,0.014651,0.314894,1.362102,7.0,1.0,0.0,1.066551,23.118222,"[2, 3]",False,False,"[1, 0]","[[1.4041287291250852, 1.458881497577924], [0.1...","[True, False]",False,"[0.5, 0.5]","[0.005778379685830558, -1.9794024411862323]","[0.26460606156025096, 0.26900607437680607]","[2.1806927519790693, 22.27990325322008]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
6,True,0.327123,2.467721,-88.800027,0.029193,0.360833,1.560818,8.0,1.0,-15.0,1.066557,23.118222,"[2, 3]",False,False,"[1, 0]","[[1.433556627918316, 1.4594318328990603], [0.1...","[True, False]",False,"[0.5, 0.5]","[0.030329816885297532, -1.949210479141305]","[0.264914072683883, 0.2685010321651815]","[2.2101241845894273, 22.32566962252279]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
7,True,0.328698,2.400251,-88.810958,0.040128,0.427430,1.848890,9.0,0.5,0.0,1.066564,23.118222,"[2, 3]",False,False,"[1, 0]","[[1.4536788314590536, 1.4601743289999483], [0....","[True, False]",False,"[0.5, 0.5]","[0.04289867821167805, -1.9296935521553935]","[0.26549085997359956, 0.2674148055382602]","[2.230251819627283, 22.35596092728149]",False,False,"[(0.3078780025243759, 2.830607533454895), (0.3..."
8,True,0.336554,2.345091,-87.278228,0.043779,0.481350,2.082122,10.0,0.5,15.0,1.066601,23.118222,"[3, 4]",False,False,"[1, 0]","[[1.484682825564017, 1.461732491442951], [0.13...","[True, False]",False,"[0.5, 0.5]","[0.0565334506109165, -1.901252324339303]","[0.2663993069025138, 0.26737289882267484]","[2.2804735385150456, 22.37041524427954]",False,False,"[(0.3078780025243759, 2.8306